In [210]:
import pandas as pd


In [211]:
#read all data
orders_header = pd.read_csv('../../data/order_header.csv')
orders_details = pd.read_csv('../../data/order_details.csv')
products = pd.read_csv('../../data/products.csv')

payment_methods = pd.read_csv('../../data/payment_methods.csv')
branches = pd.read_csv('../../data/branches.csv')
categories = pd.read_csv('../../data/categories.csv')






# Orders Headers

In [212]:
orders_header.head()

,order_id,branch_id,order_date,order_total_price,order_source,order_type,order_status
0,4fc10c2a-55a9-45d0-ad37-9e3a48d190f8,975b3d24-ce3d-4801-9c11-582a817cc591,2022-10-02 11:03:36,15.00,1,1,4
1,5babceee-5385-4fa5-b25f-4f1638a70696,975b3d24-ce3d-4801-9c11-582a817cc591,2022-10-02 10:41:05,6.80,1,1,4
2,3c811baa-93bc-4200-9204-fdee8932f020,975b3d24-ce3d-4801-9c11-582a817cc591,2022-10-02 10:39:04,5.00,1,1,4
3,7e7a67f8-849d-42da-9cac-e29d3144e9ec,975b3d24-cb71-4df8-930e-054bcd67f90c,2022-10-02 13:36:54,20.75,1,2,4
4,e02a78da-8a3e-4dad-8a27-ecc739f8d8a4,975b3d24-cb71-4df8-930e-054bcd67f90c,2022-10-02 13:15:22,30.00,1,1,4


In [ ]:
orders_types = {1:'Dine In', 2:'Pick Up', 3:'Delivery', '4':'Drive Thru'}
orders_sources = {1:'Cashier', 2:'API', 3:'Call Center'}
orders_statuses = {1:'Pending', 2:'Active', 3:'Declined', 4:'Closed', 5:'Returned', 6:'Joined', 7:'Void'}
# orders_delivery_statuses = {1:'sent to kitchen', 2:'ready', 3:'assigned', 4:'en route', 5:'delivered', 6:'closed'}
# products_statuses = {1:'Pending', 2:'Active', 3:'Closed', 4:'Moved', 5:'Void', 6:'Returned', 7:'Declined'}
# discounts_types = {1:'Open', 2:'Predefined', 3:'Coupon', 4:'Loyalty', 5:'Promotion'}

#---------------

orders_header['order_type'] = orders_header['order_type'].map(orders_types)
orders_header['order_source'] = orders_header['order_source'].map(orders_sources)
orders_header['order_status'] = orders_header['order_status'].map(orders_statuses)


In [ ]:
orders_header['order_datetime'] = pd.to_datetime(orders_header['order_date'])

orders_header['order_date'] = orders_header['order_datetime'].dt.date
orders_header['hour'] = orders_header['order_datetime'].dt.hour
orders_header['day_name'] = orders_header['order_datetime'].dt.day_name()
orders_header['is_weekend'] = orders_header['order_datetime'].dt.dayofweek.isin([4, 5])
orders_header['is_weekend'].replace({True:'Yes', False:'No'}, inplace=True)
orders_header['month'] = orders_header['order_datetime'].dt.month
orders_header['year'] = orders_header['order_datetime'].dt.year




KeyError: 'order_datetime'

In [ ]:
orders_header.drop_duplicates(inplace=True)

In [ ]:
orders_header.isnull().sum()

order_id             0
branch_id            0
order_date           0
order_total_price    0
order_source         0
order_type           0
order_status         4
hour                 0
day_name             0
is_weekend           0
month                0
year                 0
dtype: int64

In [ ]:
orders_header['order_status'].fillna('Void', inplace=True)

In [ ]:
orders_header.to_csv('../../data/processed/order_header.csv', index=False)

# Order Details

In [ ]:
orders_details.head()

,order_id,branch_id,order_date,order_total_price,order_source,order_type,order_status,product_id,category_id,quantity,product_price
0,4fc10c2a-55a9-45d0-ad37-9e3a48d190f8,975b3d24-ce3d-4801-9c11-582a817cc591,2022-10-02 11:03:36,15.0,1,1,4,975b583e-90af-4319-87d3-022d8f1936d4,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,4.5
1,4fc10c2a-55a9-45d0-ad37-9e3a48d190f8,975b3d24-ce3d-4801-9c11-582a817cc591,2022-10-02 11:03:36,15.0,1,1,4,975b583e-96ee-41d6-9ed2-c2fb747e3f0a,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,5.5
2,4fc10c2a-55a9-45d0-ad37-9e3a48d190f8,975b3d24-ce3d-4801-9c11-582a817cc591,2022-10-02 11:03:36,15.0,1,1,4,975b583e-92bd-46f1-86c1-183d3a1c3c32,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,5.0
3,5babceee-5385-4fa5-b25f-4f1638a70696,975b3d24-ce3d-4801-9c11-582a817cc591,2022-10-02 10:41:05,6.8,1,1,4,975b583e-90af-4319-87d3-022d8f1936d4,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,4.5
4,5babceee-5385-4fa5-b25f-4f1638a70696,975b3d24-ce3d-4801-9c11-582a817cc591,2022-10-02 10:41:05,6.8,1,1,4,975b583f-2d41-4563-82d1-b09078a2e80a,975b5737-cf8c-4ef3-a445-7c4a71f349c5,1,0.3


In [ ]:
filtered_orders_details = orders_details.drop(columns=['order_date', 'order_total_price','order_source', 'order_type', 'order_status', 'branch_id'])
filtered_orders_details.head()


,order_id,product_id,category_id,quantity,product_price
0,4fc10c2a-55a9-45d0-ad37-9e3a48d190f8,975b583e-90af-4319-87d3-022d8f1936d4,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,4.5
1,4fc10c2a-55a9-45d0-ad37-9e3a48d190f8,975b583e-96ee-41d6-9ed2-c2fb747e3f0a,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,5.5
2,4fc10c2a-55a9-45d0-ad37-9e3a48d190f8,975b583e-92bd-46f1-86c1-183d3a1c3c32,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,5.0
3,5babceee-5385-4fa5-b25f-4f1638a70696,975b583e-90af-4319-87d3-022d8f1936d4,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,4.5
4,5babceee-5385-4fa5-b25f-4f1638a70696,975b583f-2d41-4563-82d1-b09078a2e80a,975b5737-cf8c-4ef3-a445-7c4a71f349c5,1,0.3


In [ ]:
#filtered_orders_details['order_line_id'] = filtered_orders_details['order_id'] + '-' + filtered_orders_details['product_id'].apply(lambda x: x.split('-')[1]) + 
import uuid
filtered_orders_details['order_line_id'] = [str(uuid.uuid4()) for _ in range(len(filtered_orders_details))]

In [ ]:
filtered_orders_details.head()

,order_id,product_id,category_id,quantity,product_price,order_line_id
0,4fc10c2a-55a9-45d0-ad37-9e3a48d190f8,975b583e-90af-4319-87d3-022d8f1936d4,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,4.5,130ce1bd-2b73-4eac-b381-edf0617f89fc
1,4fc10c2a-55a9-45d0-ad37-9e3a48d190f8,975b583e-96ee-41d6-9ed2-c2fb747e3f0a,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,5.5,9dd95a69-8fda-46f4-88d0-da0d364fb870
2,4fc10c2a-55a9-45d0-ad37-9e3a48d190f8,975b583e-92bd-46f1-86c1-183d3a1c3c32,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,5.0,4f24ea83-6697-4eb4-a2b0-c16fbaeb7852
3,5babceee-5385-4fa5-b25f-4f1638a70696,975b583e-90af-4319-87d3-022d8f1936d4,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,4.5,91801e23-c702-44eb-b598-c5523c132fcb
4,5babceee-5385-4fa5-b25f-4f1638a70696,975b583f-2d41-4563-82d1-b09078a2e80a,975b5737-cf8c-4ef3-a445-7c4a71f349c5,1,0.3,adcce714-32bc-44a0-9172-25b510dfdb4a


In [ ]:
filtered_orders_details.drop_duplicates(inplace=True)

In [ ]:
filtered_orders_details[filtered_orders_details.order_line_id.duplicated()]

,order_id,product_id,category_id,quantity,product_price,order_line_id


In [ ]:
filtered_orders_details.to_csv('../../data/processed/order_details.csv', index=False)

# Products

In [ ]:
products.head()

,product_id,product_sku,product_name,category_name,category_id,is_active,is_stock_product,price
0,975b3d60-49fe-4714-a61a-927e0180f6f0,sk-002,بيبسي دايت (بيبسي دايت),مشروبات,975b3d55-4cab-406a-82a7-7a9dc5675347,True,False,0.0
1,975b3d60-4ed3-4ef7-a2ec-0e8c31e31ca5,sk-001,Pepsi (Pepsi),مشروبات,975b3d55-4cab-406a-82a7-7a9dc5675347,True,True,0.0
2,975b3d60-5c79-480a-adfd-f07452c38103,sk-003,كلاسيك لحمه (Tasbera Beef Beef),كلاسيك,975b3d56-ac1f-4e74-9447-40a0f6b1eb0b,True,False,0.0
3,975b3d60-5d3f-46ae-800b-479e5caf7b09,sk-005,بيج تشكس لحمه (بيج تشكس لحمه),بيج تشكس,975b3d57-c30a-4b98-91ed-1711c5c2ca01,True,False,0.0
4,975b3d60-6219-483a-8a07-d3a39f05df8d,sk-004,صوص ثوم حار (Chili Garlic Sauce),صوصات,975b3d54-5c9f-4134-8818-69ae552bd1d9,True,False,0.0


In [ ]:
products.to_csv('../../data/processed/products.csv', index=False)

# Branches

In [ ]:
branches.head()

,branch_id,branch_name,opening_from,opening_to
0,975b3d24-cb71-4df8-930e-054bcd67f90c,4Chicks Abdoun,09:00,03:00
1,975b3d24-ce3d-4801-9c11-582a817cc591,4Chicks 7th circle,10:00,03:00
2,976744f0-20ac-4dd5-a06a-6a1ee9ffa7b5,4Chicks Al-Jubeha,10:00,03:00


In [ ]:
branches['opening_from'] = pd.to_datetime(branches['opening_from'])
branches['opening_to'] = pd.to_datetime(branches['opening_to'])

branches['opening_from'] = branches['opening_from'].dt.hour
branches['opening_to'] = branches['opening_to'].dt.hour

In [ ]:
branches.to_csv('../../data/processed/branches.csv', index=False)

# Categories

In [ ]:
categories

,category_id,category_name
0,975b3d54-3467-4686-82ba-fe291c1cca5e,posrocket
1,975b3d54-4ce0-4438-9fcb-20e7f7861b33,تشك تشكس
2,975b3d54-5c9f-4134-8818-69ae552bd1d9,صوصات
3,975b3d54-7f8a-45d2-9b1d-07fc69a4a2f1,اضف لوجبتك
4,975b3d54-b06d-44e7-9227-62d25add8aa0,برو تشكس
5,975b3d54-edb3-4007-b56a-bcd836e98530,تشيزي شكس
6,975b3d55-0f0c-4d01-8eb9-0f13ac420574,اربيان
7,975b3d55-39e7-4019-8210-f84a9af6d86c,فرايز
8,975b3d55-43f9-46ba-8d9d-1a22c3161ab5,Burger
9,975b3d55-49f5-4ee9-945b-a22008d2e9e1,متبرقرة


In [ ]:
categories.to_csv('../../data/processed/categories.csv', index=False)

In [ ]:
# def check_values_exist(list1, list2):
#     return all(value in list2 for value in list1)

# check_values_exist(products.category_id.unique(), categories.category_id.unique())